In [ ]:
import pydot
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import math
from statistics import mean
from collections import Counter
import numpy as np
from numpy import linalg as LA
import copy
from sklearn.manifold import MDS, TSNE, trustworthiness
from matplotlib.path import Path
import matplotlib.patches as patches
from sklearn.cluster import KMeans
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import pearsonr


## Data to nodes and edges

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


def parse_dot_file(file_path):
    '''Turn dot file to nodes and edges'''
    graph = pydot.graph_from_dot_file(file_path)[0]  # Get the first graph from the list
    nodes = []
    edges = []
    for node in graph.get_nodes(): #extracts nodes
        if has_numbers(node.get_name()):
          nodes.append(node.get_name())
    for edge in graph.get_edges(): #extracts edges
        source = edge.get_source()
        target = edge.get_destination()
        attributes = edge.get_attributes()
        edges.append((source, target, attributes))

    num_nodes = len(nodes)
    '''Turn the nodes and edges to adjencecy matrix'''
    adj_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for i, edge in enumerate(edges):
        source = edge[0]
        target = edge[1]
        weight = edge[2]
        try:
            adj_matrix[nodes.index(source)][nodes.index(target)] = weight['weight']
        except KeyError:
            adj_matrix[nodes.index(source)][nodes.index(target)] = 1
    return nodes, edges, adj_matrix

    def parse_subgraph_dot_file(file_path):
      # a string version so you can play around with it easier
      graph_string = pydot.graph_from_dot_file(file_path)[0].to_string()
      graph = nx.nx_pydot.read_dot(file_path)
      # fastest way to get nodes is with networkx, pydot does not understand subgraphs
      nodes = graph.nodes()
      edges = graph.edges()

      # Remove 'n' prefix from nodes
      nodes = [node[1:] for node in nodes]

      # Remove 'n' prefix from edges
      edges = [(u[1:], v[1:]) for u, v in edges]
      return nodes, edges, graph_string


## Drawing

In [ ]:
def draw_node_link_diagram(nodes, edges, node_positions, node_colors=None, node_size=30, directed=False, title=None):
    fig, ax = plt.subplots()

    # Draw nodes and labels
    for node, pos in node_positions.items():
        if 'd' in node: #if it is a dummy
            ax.scatter(pos[0], pos[1], s=node_size/2, color="lightseagreen", edgecolors="lightseagreen", zorder=5, marker="s")  # Draw node
        elif not '|' in node:
            ax.scatter(pos[0], pos[1], s=node_size, color="teal", edgecolors="darkslategray", zorder=5)  # Draw node
        #ax.text(pos[0], pos[1], node[0:4], ha='center', va='center', fontsize=8, zorder=10)  # Add label

    #code used to add the edges not included in the tree layout to the visualization
    #nodes2, edges2, adj_matrix2 = parse_dot_file(file_path)
    #for edge in edges2:
        #if edge not in edges:
            #source, target, _ = edge
            #start = node_positions[source]
            #end = node_positions[target]
            #ax.plot([start[0], end[0]], [start[1], end[1]], color='red', linewidth=0.2, zorder=0)  # Draw edge

    # Draw edges
    for edge in edges:
        source, target, _ = edge
        start = node_positions[source]
        end = node_positions[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax.plot([start[0], end[0]], [start[1], end[1]], color='darkslategray', linewidth=0.2, zorder=0)  # Draw edge
        elif directed:
            ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/100, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color='darkslategray', linewidth=0.2, zorder=0)  # Draw edge

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title(title)
    plt.show()

In [ ]:
#slightly modified version of draw_node_link_diagram for the final layered layout visualization
def draw_node_link_diagram_s(nodes, edges, node_positions, node_colors=None, node_size=30, directed=False, title=None, bezier_curves=None):
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    # Draw nodes and labels
    for node, pos in node_positions.items():
        if not '|' in node:
            ax.scatter(pos[0], pos[1], s=node_size, color="teal", edgecolors="darkslategray", zorder=5)  # Draw node
        #ax.text(pos[0], pos[1], node[0:4], ha='center', va='center', fontsize=8, zorder=10)  # Add label

    # Draw edges
    for edge in edges:
        source, target, _ = edge
        start = node_positions[source]
        end = node_positions[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax.plot([start[0], end[0]], [start[1], end[1]], color='darkslategray', linewidth=0.2, zorder=0)  # Draw edge
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color='darkslategray', linewidth=0.2, zorder=0)  # Draw edge

    #Bonus task attempt: curved edges
    # Draw Bézier curves for edges with dummy nodes
    #if bezier_curves:
        #for bezier_curve in bezier_curves:
            #t = np.linspace(0, 1, 100)
            #curve_points = bezier_curve(t)
            #ax.plot(curve_points[0], curve_points[1], color='blue', zorder=0)  # Draw Bézier curve


    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title(title)
    plt.show()

In [ ]:
#slightly modified version of draw_node_link_diagram for projection visualizations
def draw_node_link_diagram_p(nodes, edges, node_positions, node_colors=None, node_size=30, directed=False, title=None):
    fig, ax = plt.subplots()

    for node, pos in node_positions.items():
        cluster_colors = {0: 'lightseagreen', 1: 'purple', 2: 'orange', 3: 'yellow', 4: 'darkblue', 5: 'darkgreen', 6: 'red', 7: 'black', 8: 'cyan'}  # Define cluster colors
        color = "teal" if node_colors is None else cluster_colors[node_colors[int(node)-1]]  # Default or specified node color
        if 'd' in node: #if it is a dummy
            ax.scatter(pos[0], pos[1], s=node_size/2, color=color, edgecolors="darkslategray", zorder=5, marker="s")  # Draw node
        elif not '|' in node:
            ax.scatter(pos[0], pos[1], s=node_size, color=color, edgecolors="darkslategray", zorder=5)  # Draw node

    # Draw edges
    for edge in edges:
        source, target, _ = edge
        start = node_positions[source]
        end = node_positions[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[original_source], zorder=0)  # Draw edge
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color='darkslategray', linewidth=0.2, zorder=0)  # Draw edge

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title(title)
    plt.show()

## Step 1: Basic MatplotLib Graph Drawing

In [ ]:
# Function to arrange nodes in a square-like layout
def arrange_in_square(nodes):
    num_nodes = len(nodes)
    side_length = int(math.ceil(math.sqrt(num_nodes)))
    spacing = 50
    start_x, start_y = -((side_length - 1) * spacing) / 2, ((side_length - 1) * spacing) / 2
    node_positions = {}
    current_x, current_y = start_x, start_y
    for i, node in enumerate(nodes):
        node_positions[node] = (current_x, current_y)
        current_x += spacing

        if (i + 1) % side_length == 0:
            current_x = start_x
            current_y -= spacing  # Move to the next row
    return node_positions

def arrange_in_radial(nodes):
    num_nodes = len(nodes)
    radius = 200
    angle_step = 360 / num_nodes
    node_positions = {}
    for i, node in enumerate(nodes):
        angle = math.radians(i * angle_step)
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        node_positions[node] = (x, y)
    return node_positions

def arrange_in_semi_square(nodes):
    num_nodes = len(nodes)
    side_length = int(math.ceil(math.sqrt(num_nodes)))  # Calculate the side length of the square
    spacing = 60  # Spacing between nodes
    start_x, start_y = -((side_length - 1) * spacing) / 2, ((side_length - 1) * spacing) / 2
    node_positions = {}
    current_x, current_y = start_x, start_y
    for i, node in enumerate(nodes):
        node_positions[node] = (current_x, current_y + random.randint(-15, 15))
        current_x += spacing

        if (i + 1) % side_length == 0:
            current_x = start_x
            current_y -= spacing  # Move to the next row

    return node_positions

## Step 2: Undirected Tree based algorithm + DFS&BFS

### Tree structure

In [ ]:
def arrange_in_tree(nodes, edges, root=None):
    num_nodes = len(nodes)
    node_positions = {}
    if root == None:
        root = nodes[0]

    node_positions[root] = (0, 0)
    layer_space = 1
    node_space = 1
    padding = 10

    queue = [root]
    past_nodes = []
    '''Tree creation'''
    while queue != []:
        current_node = queue[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]]
        children_nodes = [value for tuple in connected_edges for value in tuple if value != current_node and value not in past_nodes]
        if children_nodes != []:
            segment_length = node_space / len(children_nodes)
            parent_coords = node_positions[current_node]
            for i, children_node in enumerate(children_nodes):
                if children_node not in node_positions:
                    new_x = -node_space/2+segment_length*(0.5+i)
                    new_y = parent_coords[1]-layer_space
                    node_positions[children_node] = (new_x, new_y)
            queue = children_nodes + queue
        queue.remove(current_node)
        past_nodes.append(current_node)


    '''Improve node positions in tree '''
    #this part will calculate what the max and min x&y-coordinate for the graph to determine the most rightest & leftest etc. place
    layer_min_max = {}
    for node, node_pos in node_positions.items():
        layer = node_pos[1] #y-as
        x_coord = node_pos[0] #x-as
        if layer not in layer_min_max:
            layer_min_max[layer] = [x_coord, x_coord]
        else:
            if x_coord < layer_min_max[layer][0]:
                layer_min_max[layer][0] = x_coord
            if x_coord > layer_min_max[layer][1]:
                layer_min_max[layer][1] = x_coord

    # Move nodes which are already in the same layer and divide them equally through the layer
    for layer, (min_x, max_x) in layer_min_max.items():
        layer_nodes = [(node, pos) for node, pos in node_positions.items() if pos[1] == layer]
#         if len(layer_nodes) > 5:
        if len(layer_nodes) > 2: #TODO fix visual
            interval = (max_x - min_x) / (len(layer_nodes) - 1)
            for i, (node, pos) in enumerate(layer_nodes):
                new_x = min_x + i * interval
                node_positions[node] = (new_x, pos[1])
#             print(node_positions)
        elif layer_nodes[0][1][1] < 0: #check if not first layer and not a lot of nodes, connected them with their parents by giving the parent's node x-as
            for i, layer_node in enumerate(layer_nodes):
                parent_node = [parent_node[0] for parent_node in edges if parent_node[1] == layer_node[0]]
                middle_parent_node_coords = mean([parent_node_coords[1][0] for parent_node_coords in previous_layer_nodes if parent_node_coords[0] in parent_node])
                node_positions[layer_node[0]] = (middle_parent_node_coords, layer_node[1][1])
        layer_nodes = [(node, pos) for node, pos in node_positions.items() if pos[1] == layer]
        previous_layer_nodes = layer_nodes.copy()
    return node_positions

### Search Algorithms

In [ ]:
def DFS(nodes, edges, search_root=None, node_size=200):
    '''DFS'''
    num_nodes = len(nodes)
    if search_root == None:
        search_root = nodes[0]
    stack = [search_root]
    visited = []
    DFS_edges = []
    while stack != []:
        current_node = stack[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]]
        connected_edges = sorted(connected_edges, key=lambda x: int(x[1]))
        children_nodes = [value for tuple in connected_edges for value in tuple if value != current_node and value not in visited]
        if children_nodes != []:
            for children_node in children_nodes:
                if children_node in stack:
                    stack.remove(children_node)
                stack[stack.index(current_node):stack.index(current_node)] = [children_node]
            for edge in edges:
                if (edge[0] == current_node and edge[1] == children_nodes[0]
                     and edge[1] not in [elem[1] for elem in DFS_edges]): #same source and if target is first edge
                    DFS_edges.append(edge)
                elif (edge[1] == current_node and edge[0] == children_nodes[0]
                      and edge[0] not in [elem[0] for elem in DFS_edges]): #viceversa cuz undirected
                    DFS_edges.append((edge[1], edge[0], edge[2]))
        elif len(visited) != len(nodes):
            stack.insert(0,[elem[0] for elem in DFS_edges if elem[1] == current_node][0])
        if current_node not in visited:
            visited.append(current_node)
        stack.remove(current_node)

    return DFS_edges, visited

def BFS(nodes, edges, search_root=None, node_size=200):
    '''BFS'''
    BFS_edges = []
    num_nodes = len(nodes)
    if search_root == None:
        search_root = nodes[0]
    queue = [search_root]
    layers = [[search_root]]
    visited = []
    distance_matrix = np.inf * np.ones((num_nodes, num_nodes))
    while queue != []:
        current_node = queue[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]]
        connected_edges = sorted(connected_edges, key=lambda x: int(x[1]))
        parent_nodes = [value for edge in connected_edges for value in edge if value != current_node and value not in visited and current_node != edge[0]]
        children_nodes = [value for edge in connected_edges for value in edge if value != current_node and value not in visited and current_node != edge[1]]
        connected_nodes = parent_nodes + children_nodes
#         print("layers", layers, "parent", parent_nodes, "child", children_nodes, "conn", connected_nodes)
        for node in connected_nodes:
            if node not in visited:
                node_index = nodes.index(node)
                distance_matrix[nodes.index(search_root), node_index] = 1

        if connected_nodes != []:
            for connected_node in connected_nodes:
                if connected_node not in queue:
                    queue.append(connected_node)
                for edge in edges:
                    if (edge[0] == current_node and edge[1] == connected_node
                       and edge[1] not in [elem[1] for elem in BFS_edges]): #same source and if target is first edge
                        BFS_edges.append(edge)
                    # elif (edge[1] == current_node and edge[0] == connected_node
                    #       and edge[0] not in [elem[0] for elem in BFS_edges]): #viceversa cuz undirected
                    #     BFS_edges.append((edge[1], edge[0], edge[2]))
        if current_node not in visited:
            visited.append(current_node)
            for i, layer in enumerate(layers):
                if current_node in layer:
                    for connected_node in connected_nodes:
                        if connected_node in parent_nodes:
                            layers[i-1].append(connected_node) #parent
                        else:
                            try:
                                layers[i+1].append(connected_node) #child
                            except IndexError:
                                layers.append([connected_node])
        queue.remove(current_node)
        if queue == [] and len(visited) < len(nodes): #if multiple trees
            not_visited = [x for x in nodes if x not in visited]
            layers.append(['|' , not_visited[0]])
            queue = [not_visited[0]]
    return BFS_edges, visited, layers, distance_matrix



def search_through_tree(nodes, edges, node_positions, node_colors, search_root=None, node_size=200):
    print("DFS")
    DFS_edges, DFS_visited = DFS(nodes, edges, search_root)
    print('DFS Visit order', DFS_visited)
    DFS_node_positions = arrange_in_tree(nodes, DFS_edges, search_root)
    draw_node_link_diagram(DFS_visited, DFS_edges, DFS_node_positions, node_colors, node_size)
    print('DFS removed edges: ', [x[:2] for x in edges if x not in DFS_edges])
    print("BFS")
    BFS_edges, BFS_visited, layers, _ = BFS(nodes, edges, search_root)
    print('BFS Visit order', BFS_visited) #~ ADD LATER
    BFS_node_positions = arrange_in_tree(nodes, BFS_edges, search_root)
    draw_node_link_diagram(BFS_visited, BFS_edges, BFS_node_positions, node_colors, node_size)
    print('BFS removed edges: ', [x[:2] for x in edges if x not in BFS_edges]) #~ ADD LATER

## Step 3: Force directed layout

In [ ]:
def arrange_in_force(nodes, edges, k, iterations):
    # Call Fruchterman-Reingold function and return node positions
    node_positions = fruchterman_reingold(nodes, edges, iterations, k)

#     initial_node_positions = initialize_node_positions(nodes)
#     enforced_node_positions = enforce_spacing(initial_node_positions)
#     final_node_positions = apply_forces(nodes, edges, enforced_node_positions)
    return node_positions

In [ ]:
def fruchterman_reingold(nodes, edges, iterations, c, cooling_factor=0.95, temperature=1.0,):
    # Fruchterman-Reingold force-directed layout algorithm
    # Initialize node positions randomly
    node_positions = {node: (random.random(), random.random()) for node in nodes}
    k = c * math.sqrt(1 / len(nodes))

    # function to calculate repulsive force between two nodes
    def repulsive_force(pos1, pos2, k):
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        distance_squared = max(dx**2 + dy**2, 0.01)  # Avoid division by zero
        return  -k**2 / max((dx / distance_squared),0.001), -k**2 / max((dy / distance_squared),0.001)

    # function to calculate attractive force between two nodes
    def attractive_force(pos1, pos2, k):
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        distance = max(math.sqrt(dx**2 + dy**2), 0.001)  # Avoid division by zero
        return (dx / distance) / k, (dy / distance) / k

    # Perform iterations of the Fruchterman-Reingold algorithm
    for _ in range(iterations):
        # Calculate repulsive forces between all pairs of nodes
        for node1, pos1 in node_positions.items():
            node_positions[node1] = (0, 0)
            for node2, pos2 in node_positions.items():
                if node1 != node2:
                    force_x, force_y = repulsive_force(pos1, pos2, k)
                    node_positions[node1] = (node_positions[node1][0] + force_x, node_positions[node1][1] + force_y)
        # Calculate attractive forces between connected nodes
        for edge in edges:
            source, target, _ = edge
            force_x, force_y = attractive_force(node_positions[source], node_positions[target], k)
            node_positions[source] = (node_positions[source][0] + force_x, node_positions[source][1] + force_y)
            node_positions[target] = (node_positions[target][0] - force_x, node_positions[target][1] - force_y)
        # Update node positions based on forces and temperature
        for node, pos in node_positions.items():
            dx = min(abs(pos[0]), temperature) * (1 if pos[0] >= 0 else -1)
            dy = min(abs(pos[1]), temperature) * (1 if pos[1] >= 0 else -1)
            node_positions[node] = (pos[0] - dx, pos[1] - dy)
        # Cool down temperature
        temperature *= cooling_factor
    return node_positions

## Step 4: Layered Layout

In [ ]:
def get_parents(edges, node, parents=None, parent_number=None):
    if parents is None:
        parents = {}
    if parent_number is None: #parent_number is the number of layer it is above the current node
        parent_number = 0

    for edge in edges:
        if edge[1] == node:
            parent = edge[0]
            if parent not in parents:
                parent_number += 1
                parents[parent] = parent_number
                get_parents(edges, parent, parents, parent_number)
    return parents

In [ ]:
def get_childs(edges, node, childs=None, child_number=None):
    if childs is None:
        childs = {}
    if child_number is None:
        child_number = 0

    for edge in edges:
        if edge[0] == node:
            child = edge[1]
            if child not in childs:
                child_number -= 1
                childs[child] = child_number
                get_childs(edges, node, childs, child_number)

    return childs

In [ ]:
def sugiyama_framework(nodes, edges, node_colors, root=None, node_size=200):
    def guarantee_heuristic(nodes, edges, node_positions):
        '''Find the edges to be reversed to avoid cyclic graph'''
        print("Heuristic with guarantees")
        #Part 1: Sinks (only receiving edges)
        original_nodes = set(nodes) #turn the nodes to set, to use the handy set functions
        while True:
            sinks_edges = {node: [(source, target, weight) for source, target, weight in edges if target == node] for node in set([edge[1] for edge in edges]) - set([edge[0] for edge in edges])}
            removable_edges = [element for innerList in sinks_edges.values() for element in innerList]
            edges = [edge for edge in edges if edge not in removable_edges]             # Remove edges corresponding to the sink nodes
            current_nodes = set([node for edge in edges for node in edge[:2]]) #new nodes
            if current_nodes == original_nodes: # if the current set of nodes is the same as the original set, exit loop
                break
            else: # otherwise, update the set of nodes for the next iteration
                original_nodes = current_nodes
                nodes = list(original_nodes) #turn it back to the original list type
        print("Removed sinks:")
        draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)


        #Part 2: Sources (only outcoming edges)
        original_nodes = set(nodes) #turn the nodes to set, to use the handy set functions
        while True:
            sources_edges = {node: [(source, target, weight) for source, target, weight in edges if source == node] for node in set([edge[0] for edge in edges]) - set([edge[1] for edge in edges])}
            removable_edges = [element for innerList in sources_edges.values() for element in innerList]
            edges = [edge for edge in edges if edge not in removable_edges]
            current_nodes = set([node for edge in edges for node in edge[:2]])
            if current_nodes == original_nodes:
                break
            else:
                original_nodes = current_nodes
                nodes = list(original_nodes)
        print("Removed sources:")
        draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

        #Part 3: Inversing the edges
        # Create a Counter to count the occurrences of each node in the edge structure (+1 if it a source -1 if it a target)
        node_counter = Counter()
        for source, target, weight in edges:
            node_counter[source] += 1 # N ->
            node_counter[target] -= 1 # N <-
        nodes = sorted(node_counter, key=node_counter.get, reverse=True) #get counter in reversed aka descending order
        cycle_edges = []
        for node in nodes:
            temp_edges = []
            for edge in edges:
                if edge[0] != node or edge in cycle_edges:
                    temp_edges.append(edge)
                if edge[1] == node:
                    cycle_edges.append(edge)
            edges = temp_edges

        print("Found cycle edges")
        draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)
        return edges

    def layer_assignment(nodes, edges, node_positions, root=None):
        #add to layers from root
        if root == None:
            root = nodes[0]
        print("nodes, edges, root", nodes, edges, root)
        BFS_edges, BFS_visited, layers, _ = BFS(nodes, edges, root)
        print('BFS Visit order', BFS_visited)
        print('BFS removed edges: ', [x[:2] for x in edges if x not in BFS_edges])
        node_positions = {}
        layer_counter = 0
        node_space = 0
        placed_nodes_in_layers = copy.deepcopy(layers)
        for index, layer in enumerate(layers):
            if "|" in layer:
                layer_counter = 0
            for node in layer: #loops through the layers
                if node not in node_positions:  #if the first time encountering a node place it
                    node_positions[node] = (node_space, layer_counter)
                    node_space += 1
                else: #if not, then also move every parent from that node to the current position of the node
                    node_parents = get_parents(edges, node)
                    for node_parent, parent_layer in node_parents.items():
                        if node_parent in node_positions:
                            node_positions[node_parent] = (node_positions[node_parent][0], node_positions[node][1] + parent_layer)
                        else:
                            node_positions[node_parent] = (node_space, node_positions[node][1] + parent_layer)
            layer_counter -= 1

        # #Adjusting the nodes which have parents/childs that have only dummies in between them
        for node, (x, y) in node_positions.items():
            node_childs = [key for key, value in get_childs(edges, node).items() if value <= 2] #key is node, value is the node_positions
            node_parents = [key for key, value in get_parents(edges, node).items() if value <= 2]
            if list(node_childs) != []:
                if y-1 != max(node_positions[node_child][1] for node_child in node_childs):
                    #if the max y-as of a child isn't directly under the current node
                    node_positions[node] = (x, y-1) #move it down
            if list(node_parents) != []:
                if y+1 != min(node_positions[node_parent][1] for node_parent in node_parents):
                    #if the min y-as of a parent isn't directly above the current node
                    node_positions[node] = (x, y+1) #move it up

        #Add dummies on graph
        layer_crossing_edges = [edge for edge in edges if abs(node_positions[edge[0]][1] - node_positions[edge[1]][1]) >= 2]
        dummy_number = 0
        while layer_crossing_edges != []:
            for index, edge in enumerate(layer_crossing_edges):
                dummy_number += 1
                dummy_name = 'd'+str(dummy_number)
                nodes.append(dummy_name) #add dummy to node list
                node_colors[dummy_name] = "#808080" #add its color
                node_positions[dummy_name] = (((node_positions[edge[0]][0] + node_positions[edge[1]][0])/2), round((node_positions[edge[0]][1] + node_positions[edge[1]][1])/2))
                edges.append((edge[0], dummy_name, edge[2]))
                edges.append((dummy_name, edge[1], edge[2]))
                edges.remove(edge)
            layer_crossing_edges = [edge for edge in edges if abs(node_positions[edge[0]][1] - node_positions[edge[1]][1]) >= 2]

        return nodes, edges, node_positions

    def crossing_minimization(nodes, edges, node_positions, method="Barycenter"):
        layers = sorted(set(y for _, (_, y) in node_positions.items()), reverse=True)

        def implement_new_x (node, node_positions, barycenter_x, neighbors):
            node_positions[node] = (barycenter_x, node_positions[node][1])
            return node_positions

        def calculate_barycenter(node, parents, node_positions, neighbors):
            parents_x_sum = sum(node_positions[parent][0] for parent in parents)
            if parents_x_sum == 0:
                barycenter_x = node_positions[node][0]
            else:
                barycenter_x = parents_x_sum / len(parents)
            node_positions = implement_new_x (node, node_positions, barycenter_x, neighbors)
            return node_positions

        #calculate first barycenter
        for layer in layers[:-1]:
            nodes_in_curr_layer = [node for node, (x, y) in node_positions.items() if y == layer]
            nodes_in_next_layer = [node for node, (x, y) in node_positions.items() if y == layer-1]
            has_parents = {node: [] for node in nodes_in_next_layer}
            for node in nodes_in_next_layer:
                for source, target, _ in edges:
                    if target == node:
                        has_parents[node].append(source)
                node_positions = calculate_barycenter(node, has_parents[node], node_positions, nodes_in_next_layer)

            #recoordinate too close nodes
            node_groups = {}
            for node, pos in node_positions.items():
                x_coord, y_coord = pos
                if (x_coord, y_coord) in node_groups:
                    node_groups[(x_coord, y_coord)].append(node)
                else:
                    node_groups[(x_coord, y_coord)] = [node]
            too_close_groups = [group for group in node_groups.values() if len(group) > 1 and max(pos[0] for node in group) - min(pos[0] for node in group) < 1.2]
            # Divide each too close group with a spacing of 1
            for group in too_close_groups:
                group_size = len(too_close_groups)
                if group_size % 2 != 0: #if odd then middle node is middle
                    step_start = -(group_size-1)/2
                    for i, node in enumerate(group):
                        node_positions[node] = (node_positions[node][0] + step_start+i,  node_positions[node][1])
                else: #else 2 nodes are the middle
                    step_start = -(group_size)/2+0.6
                    for i, node in enumerate(group):
                        node_positions[node] = (node_positions[node][0] + step_start+i,  node_positions[node][1])

        return nodes, edges, node_positions

    #Bonus task: node positioning
    def node_positioning(nodes, edges, node_positions):
      border = node_positions['|'][0]-0.5
      tree1 = {node: pos for node, pos in node_positions.items() if pos[0] < border}
      tree2 = {node: pos for node, pos in node_positions.items() if pos[0] >= border}

      def straighten_edges(tree_positions, border, edges):
          layers = sorted(set(y for _, (_, y) in tree_positions.items()))
          max_nodes_layer = max(set( [pos[1] for node, pos in tree_positions.items()]), key= [pos[1] for node, pos in tree_positions.items()].count)
          counter = 0
          tree_positions = {node: (counter := counter + 1.2, y) if y == max_nodes_layer else (x, y) for node, (x, y) in tree_positions.items()}
          for layer in layers:
            if layer < max_nodes_layer:
              for node in tree_positions.keys():
                if tree_positions[node][1] == layer:
                  parents=get_parents(edges, node)
                  direct_parents = [node_positions[key][0] for key, value in parents.items() if tree_positions[key][1] == layer+1]
                  middle_coords = mean(direct_parents)
                  tree_positions[node] = (middle_coords, tree_positions[node][1])
            elif layer > max_nodes_layer:
              for node in tree_positions.keys():
                if tree_positions[node][1] == layer:
                  childs=get_childs(edges, node)
                  direct_childs = [node_positions[key][0] for key, value in childs.items() if tree_positions[key][1] == layer-1]
                  middle_coords = mean(direct_childs)
                  tree_positions[node] = (middle_coords, tree_positions[node][1])
          tree_positions['2'] = (3,tree_positions['2'][1])
          tree_positions['10'] = (7,tree_positions['10'][1])
          tree_positions['5'] = (1,tree_positions['5'][1])
          tree_positions['7'] = (2,tree_positions['5'][1])
          tree_positions['24'] = (5,tree_positions['5'][1])
          tree_positions['17'] = (6,tree_positions['5'][1])
          tree_positions['d1'] = (7,tree_positions['5'][1])
          return tree_positions

      tree1 = straighten_edges(tree1, border, edges)
      return {**tree1, **tree2}

    def edge_drawing(nodes, edges, node_positions):
      return None

    node_positions = {node: (random.random(), random.random()) for node in nodes}
    print("Original Graph:")
    draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

    cyclic_edges = guarantee_heuristic(nodes, edges, node_positions)
    #reverse the cyclic_edges using guarantee_heuristic
    edges = [(edge[1], edge[0], edge[2]) if edge in cyclic_edges else edge for edge in edges]

    print("Layer Assignment")
    #Improve arrow layers by assigning dummies
    nodes, edges, node_positions = layer_assignment(nodes, edges, node_positions, root)
    draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

    print("Crossing minimization")
    #Barycenter Heuristic:
    nodes, edges, node_positions = crossing_minimization(nodes, edges, node_positions, method='Barycenter')
    draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

    #Bonus task: node positioning
    #print("Node Positioning")
    #node_positions = node_positioning(nodes, edges, node_positions)
    #draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

    print("Edge Drawing")
    draw_node_link_diagram_s(nodes, edges, node_positions, node_colors, node_size)
    # print("node_positions", node_positions)
    # print("Smooth Edge Drawing")
    # draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size, last=True, smooth=True)

    return node_positions

## Step 5: Multilayer Graph

In [ ]:
def arrange_in_subgraphs(nodes, edges, graph):
    subgraphs = graph.split("subgraph")
    # in case of argumentatin network: subgraph "Youngest Devonian Strata" and subgraph "Gap in the Sequence of Devonshi"
    subgraph1 = subgraphs[6]
    subgraph2 = subgraphs[7]

    # per subgraph, get the corresponding nodes and edges
    sg1_nodes = []
    sg2_nodes = []

    for node in nodes:
        if node in subgraph1:
            sg1_nodes.append(node)
        else:
            sg2_nodes.append(node)

    sg1_edges = []
    sg2_edges = []

    for edge in edges:
        if edge[0] in sg1_nodes and edge[1] in sg1_nodes:
            sg1_edges.append(edge)
        if edge[0] in sg2_nodes and edge[1] in sg2_nodes:
            sg2_edges.append(edge)

    node_positions1 = arrange_in_force_subgraphs(sg1_nodes,sg1_edges, iterations=300, k=0.8, subgraph=0)
    node_positions2 = arrange_in_force_subgraphs(sg2_nodes,sg2_edges, iterations=300, k=0.8, subgraph=2)

    inter_connecting_edges = [(source, target) for source, target in edges if ((source in sg1_nodes and target in sg2_nodes) or (source in sg2_nodes and target in sg1_nodes))]
    inter_connecting_edges_bundles = edge_bundling(inter_connecting_edges, node_positions1, node_positions2)

    draw_subgraphs_node_link_diagram(sg1_edges, sg2_edges, inter_connecting_edges, inter_connecting_edges_bundles, node_positions1, node_positions2, node_size=50, directed=False)
    return node_positions1, node_positions2, inter_connecting_edges

In [ ]:
def arrange_in_force_subgraphs(nodes, edges, iterations, k, subgraph):
    node_positions = fruchterman_reingold_subgraphs(nodes, edges, iterations, k, subgraph)
    return node_positions

In [ ]:
def fruchterman_reingold_subgraphs(nodes, edges, iterations, k, subgraph, temperature=1.0, cooling_factor=0.8):
    # Fruchterman-Reingold force-directed layout algorithm
    # Initialize node positions randomly
    node_positions = {node: (random.uniform(0 + subgraph, 1 + subgraph), random.uniform(0 + subgraph, 1 + subgraph)) for node in nodes}

    # function to calculate repulsive force between two nodes
    def repulsive_force(pos1, pos2, k):
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        distance_squared = max(dx**2 + dy**2, 0.01)  # Avoid division by zero
        return -k**2 / max((dx / distance_squared),0.001), -k**2 / max((dy / distance_squared),0.001)

    # function to calculate attractive force between two nodes
    def attractive_force(pos1, pos2, k):
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        distance = max(math.sqrt(dx**2 + dy**2), 0.01)  # Avoid division by zero
        return (dx / distance) / k, (dy / distance) / k


    # Perform iterations of the Fruchterman-Reingold algorithm
    for _ in range(iterations):
        # Calculate repulsive forces between all pairs of nodes
        for node1, pos1 in node_positions.items():
            node_positions[node1] = (0 + subgraph, 1 + subgraph)
            for node2, pos2 in node_positions.items():
                if node1 != node2:
                    force_x, force_y = repulsive_force(pos1, pos2, k)
                    node_positions[node1] = (node_positions[node1][0] + force_x, node_positions[node1][1] + force_y)
        # Calculate attractive forces between connected nodes
        for edge in edges:
            source, target = edge
            force_x, force_y = attractive_force(node_positions[source], node_positions[target], k)
            node_positions[source] = (node_positions[source][0] + force_x, node_positions[source][1] + force_y)
            node_positions[target] = (node_positions[target][0] - force_x, node_positions[target][1] - force_y)
        # Update node positions based on forces and temperature
        for node, pos in node_positions.items():
            dx = min(abs(pos[0]), temperature) * (1 if pos[0] >= 0 else -1)
            dy = min(abs(pos[1]), temperature) * (1 if pos[1] >= 0 else -1)
            node_positions[node] = (pos[0] - dx, pos[1] - dy)
        # Cool down temperature
        temperature *= cooling_factor

    norm = []
    for node, pos in node_positions.items():
        norm.append(node_positions[node])
    norm = normalise(norm)

    for i, node in enumerate(nodes):
        node_positions[node] = (norm[i][0] + subgraph, norm[i][1] + subgraph)

    return node_positions

In [ ]:
# Normalize, definition from https://www.loekvandenouweland.com/content/normalise-list-of-tuples-in-python.html
def normalise(matrix, bundle=False):
    transposed = [[row[col] for row in matrix] for col, _ in enumerate(matrix[0])]
    normalised = [[(v - min(r)) / (max(r) - min(r)) for v in r] for r in transposed]
    return [tuple([row[col] for row in normalised]) for col, _ in enumerate(normalised[0])]

In [ ]:
def edge_bundling(edges, node_positions1, node_positions2):
    # connected edges between subgraphs only

    # l = current length of one segment
    # n = number of segments
    def compatibility(p,q,n):
        p_len = distance.euclidean(p[0],p[n])
        q_len = distance.euclidean(q[0],q[n])
        l_avg = (p_len+q_len) / 2

        # Angle
        p_vector = [(p[0][0]-p[1][0]), (p[0][1]-p[1][1])]
        q_vector = [(q[0][0]-p[1][0]), (q[0][1]-p[1][1])]
        dot_product = p_vector[0]*q_vector[0]+p_vector[1]*q_vector[1]
        magnitude_p = math.sqrt(p_vector[0]**2 + p_vector[1]**2)
        magnitude_q = math.sqrt(q_vector[0]**2 + q_vector[1]**2)
        if magnitude_p > 0 and magnitude_q > 0:
            c_a = 0.5 * (1 + dot_product / (magnitude_p * magnitude_q))
        else:
            c_a = 0


        # Scale
        c_s = 2 / (l_avg * min(p_len,q_len)+max(p_len,q_len)/l_avg)

        # Distance
        p_m = (p[0][0]+((p[n][0]-p[0][0])/2), p[0][1]+((p[n][1]-p[0][1])/2))
        q_m = (q[0][0]+((q[n][0]-q[0][0])/2), q[0][1]+((q[n][1]-q[0][1])/2))
        c_p = l_avg / (l_avg + math.sqrt((p_m[0]-q_m[0])**2+(p_m[1]-q_m[1])**2))

        # Visibility
        i0_pq = (q[0][0], p[0][1] + (q[0][0] - p[0][0]) / (p[n][0] - p[0][0]) * (p[n][1] - p[0][1]))
        i1_pq = (q[n][0], p[0][1] + (q[n][0] - p[0][0]) / (p[n][0] - p[0][0]) * (p[n][1] - p[0][1]))
        i_m_pq = ((i0_pq[0]+i1_pq[0])/2, (i0_pq[1]+i1_pq[1])/2)
        vis_pq = max(1-((2*math.sqrt((p_m[0]-i_m_pq[0])**2+(p_m[1]-i_m_pq[1])**2))/math.sqrt((i0_pq[0]-i1_pq[0])**2+(i0_pq[1]-i1_pq[1])**2)),0)
        i0_qp = (p[0][0], q[0][1] + (p[0][0] - q[0][0]) / (q[n][0] - q[0][0]) * (q[n][1] - q[0][1]))
        i1_qp = (p[n][0], q[0][1] + (p[n][0] - q[0][0]) / (q[n][0] - q[0][0]) * (q[n][1] - q[0][1]))
        i_m_qp = ((i0_qp[0]+i1_qp[0])/2, (i0_qp[1]+i1_qp[1])/2)
        vis_qp = max(1-((2*math.sqrt((q_m[0]-i_m_qp[0])**2+(q_m[1]-i_m_qp[1])**2))/math.sqrt((i0_qp[0]-i1_qp[0])**2+(i0_qp[1]-i1_qp[1])**2)),0)
        c_v = min(vis_pq,vis_qp)

        # Combined
        c_e = c_a * c_s * c_p * c_v
        return c_e

    def spring_force(p1,segment,l,n):
        if segment > 1:
            prev = math.sqrt((p1[segment-1][0]-p1[segment][0])**2+(p1[segment-1][1]-p1[segment][1])**2)
        else:
            prev = 0
        if segment <= n-2:
            if n == 10:
                print(f"list is {p1}")
            plus = math.sqrt((p1[segment][0]-p1[segment+1][0])**2+(p1[segment][1]-p1[segment+1][1])**2)
        else:
            plus = 0
        return l * (prev+plus)

    def  electrostatic_force(p1,q1,comp):
        return comp / math.sqrt((p1[0]-q1[0])**2+(p1[1]-q1[1])**2)

    node_positions = {**node_positions2, **node_positions1}
    step_size = 0.01


    n = 5
    c = 0
    cycles = 3
    while c < cycles:
      for edge in edges:
          con_edges = []
          segments = []
          start = node_positions[edge[0]]
          end = node_positions[edge[1]]
          x_diff = end[0] - start[0]
          y_diff = end[1] - start[1]
          segments.append(start)
          i = 1
          while i < n:
              segments.append((start[0] + ((x_diff/n)*i), start[1] + ((y_diff/n)*i)))
              i += 1
          segments.append(end)
          l = math.sqrt(x_diff**2 + y_diff**2)/n
          con_edges.append((segments,l))


      for p in con_edges:
          segment = 1
          while segment < n:
              fs = spring_force(p[0], segment, p[1],n)
              p[0][segment] = (p[0][segment][0] + (fs*step_size), p[0][segment][1] + (fs*step_size))
              segment += 1
          for q in con_edges:
              if p != q:
                  comp = compatibility(p[0],q[0],n)
                  segment_pair = 1
                  while segment_pair < n:
                      fe = electrostatic_force(p[0][segment_pair],q[0][segment_pair],comp)
                      p[0][segment_pair] = (p[0][segment_pair][0] + (fe*step_size), p[0][segment_pair][1] + (fe*step_size))
                      segment_pair += 1
      c += 1
      n = n*2
      step_size = step_size/2

    return con_edges;

In [ ]:
# FOR SUBGRAPHS
def draw_subgraphs_node_link_diagram(edges1, edges2, con_nodes, con_edges, node_positions1, node_positions2, node_size=10, directed=False):
    fig, ax = plt.subplots()

    # Draw nodes and labels
    for node, pos in node_positions1.items():
        ax.scatter(pos[0], pos[1], s=node_size, color='orangered', edgecolors='dimgray', zorder=5)  # Draw node

    for node, pos in node_positions2.items():
        ax.scatter(pos[0], pos[1], s=node_size, color='orangered', edgecolors='dimgray', zorder=5)

    # Draw edges
    for edge in edges1:
        source, target = edge
        start = node_positions1[source]
        end = node_positions1[target]
        if directed:
             ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color='dimgray', zorder=0)  # Draw edge

    for edge in edges2:
        source, target = edge
        start = node_positions2[source]
        end = node_positions2[target]
        if directed:
             ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color='dimgray', zorder=0)


    for node,edge in zip(con_nodes,con_edges):
        sequence = edge[0]
        x = []
        y = []
        i = 0
        while i < len(sequence):
            x.append(sequence[i][0])
            y.append(sequence[i][1])
            i += 1
        source, target = node
        if directed:
            ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax.plot(x, y, color='rosybrown', zorder=0)

    plt.hlines(y = -0.25, xmin = -0.25, xmax = 1.25, color='black')
    plt.hlines(y = 1.25, xmin = -0.25, xmax = 1.25, color='black')
    plt.vlines(x = -0.25, ymin = -0.25, ymax = 1.25, color='black')
    plt.vlines(x = 1.25, ymin = -0.25, ymax = 1.25, color='black')

    plt.hlines(y = 1.75, xmin = 1.75, xmax = 3.25, color='black')
    plt.hlines(y = 3.25, xmin = 1.75, xmax = 3.25, color='black')
    plt.vlines(x = 1.75, ymin = 1.75, ymax = 3.25, color='black')
    plt.vlines(x = 3.25, ymin = 1.75, ymax = 3.25, color='black')

#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
    fig.set_size_inches(18, 10, forward=True)
    plt.show()

In [ ]:
# Try out
#one, two, con_edges = arrange_in_subgraphs(nodes, edges, G)

## Step 6: Projections for graphs

In [ ]:
import networkx as nx

def compute_similarity_matrix(nodes, edges):
    # Construct graph from edges
    G = nx.Graph()
    for u, v, attrs in edges:
        if 'weight' in attrs:
            weight = float(attrs['weight'])  # Extract weight from edge attributes
        else:
            goal_difference = attrs['goal_difference'] #in the case of directed graphs - goal difference instead of weight
            if goal_difference.startswith('"') and goal_difference.endswith('"'): #check if it is a negative number
                goal_difference = goal_difference[1:-1]  #remove surrounding quotes from negative numbers
            weight = float(goal_difference)
        G.add_edge(u, v, weight=weight)
    # Compute shortest path distances
    shortest_path_lengths = nx.floyd_warshall_numpy(G, weight='weight')
    print(shortest_path_lengths.shape)
    print(shortest_path_lengths)
    # Convert distances to similarities
    max_distance = np.max(shortest_path_lengths[np.isfinite(shortest_path_lengths)])
    similarity_matrix = 1 - shortest_path_lengths / max_distance

    return similarity_matrix


In [ ]:
def graph_projection(nodes, edges, iter, lr, p, eps):
    # Compute similarity matrix
    similarity_matrix = compute_similarity_matrix(nodes, edges)
    dissimilarity_matrix = 1 - similarity_matrix.copy()  # Convert similarities to dissimilarities

    print(similarity_matrix)
    # Perform MDS with similarity matrix
    mds = MDS(n_components=2, dissimilarity='precomputed', eps=eps, max_iter=iter, normalized_stress='auto')
    node_positions_mds = mds.fit_transform(similarity_matrix)

    # Perform t-SNE with distance matrix
    tsne = TSNE(n_components=2, init="random", metric='precomputed', n_iter=iter, learning_rate=lr, perplexity=p)
    node_positions_tsne = tsne.fit_transform(dissimilarity_matrix)

    # Cluster the projections
    kmeans_mds = KMeans(n_clusters=8, random_state=42, n_init='auto')
    labels_mds = kmeans_mds.fit_predict(node_positions_mds)

    kmeans_tsne = KMeans(n_clusters=8, random_state=42, n_init='auto')
    labels_tsne = kmeans_tsne.fit_predict(node_positions_tsne)

    node_positions_mds_dict = {str(i+1): tuple(pos) for i, pos in enumerate(node_positions_mds)}
    node_positions_tsne_dict = {str(i+1): tuple(pos) for i, pos in enumerate(node_positions_tsne)}

    mds_title = f"MDS Diagram (max_iterations={iter}, stress threshold={eps})"
    draw_node_link_diagram_p(nodes, edges, node_positions_mds_dict, title=mds_title, node_colors=labels_mds)
    crossings, smallest_angle = count_edge_crossings(edges, node_positions_mds_dict)
    print("Number of edge crossings:", crossings, "smallest angle:", smallest_angle)
    layout_stress = compute_stress(nodes, edges, node_positions_mds_dict)
    print("Layout Stress:", layout_stress)
    print('Trustworthiness', trustworthiness(similarity_matrix, node_positions_mds))
    print('Continuity', trustworthiness(node_positions_mds, similarity_matrix))
    stress = mds.stress_
    print("Stress:", stress)
    draw_shepard_diagram(node_positions_mds, similarity_matrix)

    tsne_title = f"TSNE Diagram (iterations={iter}, learning rate ={lr}, perplexity={p})"
    draw_node_link_diagram_p(nodes, edges, node_positions_tsne_dict, title=tsne_title, node_colors=labels_tsne)
    crossings, smallest_angle = count_edge_crossings(edges, node_positions_tsne_dict)
    print("Number of edge crossings:", crossings, "smallest angle:", smallest_angle)
    layout_stress = compute_stress(nodes, edges, node_positions_tsne_dict)
    print("Stress:", layout_stress)
    draw_shepard_diagram(node_positions_tsne, similarity_matrix)
    print('Trustworthiness', trustworthiness(similarity_matrix, node_positions_tsne))
    print('Continuity', trustworthiness(node_positions_tsne, similarity_matrix))
    stress = tsne.kl_divergence_
    print("Stress:", stress)

    return node_positions_mds_dict, node_positions_tsne_dict


##Step 7: Quality Metrics

In [ ]:
def count_edge_crossings(edges, node_positions):
    crossings = 0
    smallest_angle = float('inf')  # Initialize with infinity

    for i, edge1 in enumerate(edges):
        x1, y1 = node_positions[edge1[0]]
        x2, y2 = node_positions[edge1[1]]

        # Skip if the edge ends at the same node
        if edge1[0] == edge1[1]:
            continue

        for j, edge2 in enumerate(edges[i + 1:]):
            x3, y3 = node_positions[edge2[0]]
            x4, y4 = node_positions[edge2[1]]

            # Skip if the edges start or end at the same node
            if edge1[0] == edge2[0] or edge1[0] == edge2[1] or edge1[1] == edge2[0] or edge1[1] == edge2[1]:
                continue

            # Calculate the orientation of three points (p, q, r)
            def orientation(p, q, r):
                val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
                if val == 0:
                    return 0  # Collinear
                return 1 if val > 0 else -1  # Clockwise or counterclockwise

            # Check if edges intersect
            if orientation((x1, y1), (x2, y2), (x3, y3)) != orientation((x1, y1), (x2, y2), (x4, y4)) \
                    and orientation((x3, y3), (x4, y4), (x1, y1)) != orientation((x3, y3), (x4, y4), (x2, y2)):
                crossings += 1

                # Calculate the angle between the edges
                angle1 = math.atan2(y2 - y1, x2 - x1)
                angle2 = math.atan2(y4 - y3, x4 - x3)
                angle = abs(angle1 - angle2)
                if angle > math.pi:
                    angle = 2 * math.pi - angle
                if angle < smallest_angle:
                    smallest_angle = angle

    return crossings, smallest_angle

In [ ]:
#calculate the graph theoretic distance
#similar to previous case, except without similarity matrix
def compute_graph_theoretic_distance(nodes, edges):
    non_dummy_nodes = [node for node in nodes if not node.startswith('d')]
    G2 = nx.Graph()
    for u, v, attrs in edges:
        if 'weight' in attrs:
            weight = float(attrs['weight'])  #get weight from edge attributes
        else:
            goal_difference = attrs['goal_difference'] #in the case of directed graphs - goal difference instead of weight
            if goal_difference.startswith('"') and goal_difference.endswith('"'): #check if it is a negative number
                goal_difference = goal_difference[1:-1]  #remove surrounding quotes from negative numbers
            weight = float(goal_difference)
        G2.add_edge(u, v, weight=weight)

    shortest_path_lengths = nx.floyd_warshall_numpy(G2, weight='weight')
    return shortest_path_lengths

#calculate the euclidean distance matrix
def compute_euclidean_distance_matrix(node_positions):
    non_dummy_nodes = {node: pos for node, pos in node_positions.items() if not node.startswith('d')}
    node_to_index = {node: i for i, node in enumerate(non_dummy_nodes)}
    num_nodes = len(non_dummy_nodes)
    distance_matrix = np.zeros((num_nodes, num_nodes))

    for u, v, _ in edges:
        if u.startswith('d') or v.startswith('d'):
            continue
        index1 = node_to_index[u]
        index2 = node_to_index[v]
        x1, y1 = node_positions[u]
        x2, y2 = node_positions[v]
        d_ij = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        distance_matrix[index1, index2] = d_ij
        distance_matrix[index2, index1] = d_ij

    return distance_matrix

#calculate the stress of layout
def compute_stress(nodes, edges, node_positions):
    graph_distance = compute_graph_theoretic_distance(nodes, edges)
    print('graph distance:', graph_distance)
    euclidean_distance = compute_euclidean_distance_matrix(node_positions)
    print('euclidean distance:', euclidean_distance)
    numerator = np.sum((euclidean_distance - graph_distance)**2)
    denominator = np.sum(graph_distance**2)
    return np.sqrt(numerator / denominator)

In [ ]:
def draw_shepard_diagram(node_positions, similarity_matrix):
    embedded_distances = pairwise_distances(node_positions, metric='euclidean')

    original_distances = np.sqrt(2 * (1 - similarity_matrix)) #original distances from the similarity matrix

    #flattened for plotting
    embedded_distances_flat = embedded_distances.flatten()
    original_distances_flat = original_distances.flatten()

    #create plot
    plt.scatter(original_distances_flat, embedded_distances_flat, alpha=0.5)
    plt.xlabel('Original Distances')
    plt.ylabel('Embedded Distances')
    plt.plot(original_distances_flat, original_distances_flat, color='red', linestyle='--', label='Perfect Fit')
    plt.title('Shepard Diagram')
    plt.show()

    #calculate the pearson correlation coefficient for the goodness of fit metric
    pearson_corr, _ = pearsonr(original_distances_flat, embedded_distances_flat)
    print("Pearson correlation coefficient:", pearson_corr)

## Main

In [ ]:
# Main function to draw the node-link diagram using Matplotlib
def main(nodes, edges, structure='square', node_size=300, root=None, search_algorithm=False, search_root=None, directed=False, k=0.1, iterations=500, learning_rate=200, perplexity=30, episodes=0.001, g=''):
    #add random color to each node (as node:colorvalue)
    node_colors = {}
    for node in nodes:
        node_colors[node] = "#" + ''.join(random.choices('0123456789ABCDEF', k=6)) #create random 6 hex code for color (which networkx uses)

    # Arrange nodes in a structure
    if structure =='square':
        node_positions = arrange_in_square(nodes)
    elif structure == 'radial':
        node_positions = arrange_in_radial(nodes)
    elif structure == 'semi_square':
        node_positions = arrange_in_semi_square(nodes)
    elif structure == 'tree':
        if directed:
            node_positions = sugiyama_framework(nodes, edges, node_colors, root, node_size)
        else:
            node_positions = arrange_in_tree(nodes, edges, root)
    elif structure == 'force':
        node_positions = arrange_in_force(nodes,edges, k, iterations)
    elif structure == 'subgraph':
        node_positions1, node_positions2, con_edges = arrange_in_subgraphs(nodes, edges, g)
    elif structure == 'projection':
        node_positions_mds, node_positions_tsne = graph_projection(nodes, edges, int(iterations), int(learning_rate), int(perplexity), float(episodes))
    else:
        print('Structure typed incorrectly (square, radial, semi_square or tree) OR chosen structure not undirected')

    if search_algorithm and structure == 'tree':
        search_through_tree(nodes, edges, node_positions, node_colors, search_root, node_size)


    if structure != 'tree' and structure != 'projection':
      draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size, directed)

    if structure != 'projection':
      crossings, smallest_angle = count_edge_crossings(edges, node_positions)
      print("Number of edge crossings:", crossings, "smallest angle:", smallest_angle)
      layout_stress = compute_stress(nodes, edges, node_positions)
      print("Stress:", layout_stress)


## Run Templates

In [ ]:
# '''Step 1: Basic Graph Drawing'''
# file_path = "/content/LesMiserables.dot"
# nodes, edges, adj_matrix = parse_dot_file(file_path)
# main(nodes, edges, "square")

In [ ]:
# '''Step 2: Les Miserables Tree'''
# file_path = "/content/LesMiserables.dot"
# nodes, edges, adj_matrix = parse_dot_file(file_path)
# main(nodes, edges, "tree", node_size=10, root='1', search_algorithm=True, search_root='11')

In [ ]:
# '''Step 3: Les Forced Directed Layout'''
# file_path = "DataSet/LesMiserables.dot"
# nodes, edges, adj_matrix = parse_dot_file(file_path)
# main(nodes, edges, "force", node_size=30, directed=True, k=0.08, iterations=100)

In [ ]:
# '''Step 4: Layered layout'''
# file_path = "DataSet/noname.dot"
# nodes, edges, adj_matrix = parse_dot_file(file_path)
# main(nodes, edges, 'tree', node_size = 30, directed=True)

In [ ]:
# '''Step 5: Subgraphs'''
# file_path = "DataSet/devonshiredebate_withclusters.dot"
# nodes, edges, G = parse_subgraph_dot_file(file_path)
# main(nodes, edges, 'subgraph', node_size = 30, directed=True, g=G)

In [ ]:
# '''Step 6: Projections for graphs'''
#file_path = "DataSet/JazzNetwork.dot" #LesMiserables & LeagueNetwork & JazzNetwork
#nodes, edges, adj_matrix = parse_dot_file(file_path)
# main(nodes, edges, "projection", iterations=1000, episodes=0.0001, learning_rate=200, perplexity=10)

## GUI

In [ ]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

In [ ]:
# List of step parameters
step1 = [['square', 'radial', 'semi_square'], 200, None, False, None, False, 0.1, 1000, 500, 200, 30, 0.001]
step2 = ['tree', 200, '1', True, ["Give node to use on search_algorithm (None becomes first)"], False, 0.1, 1000, 200, 30, 0.001]
step3 = ['force',200, None, False, None, True, ["Give K value between 0 and 0.5"], ["Give iteration value between 100 and 10000"], 200, 30, 0.001]
step4 = ['tree', 200, None, False, None, True, 0.1, 1000, 200, 30, 0.001]
#Step 5 Not implemented
step6 = ['projection', 200,None, False, None, True, 0.1, ["Give iteration value between 100 and 10000 (750)"], ["Give learning rate value between 150 and 300 (200)"], ["Give perplexity value between 10 and 30 (30)"], ["Give Stress convergence value between 0.00001 and 0.001 (0.00001)"]]

# List of file options
file_options = {'devonshiredebate':[step4], 'JazzNetwork':[step1, step2, step3], 'LeagueNetwork':[step4, step6], 'LesMiserables':[step1, step2, step3, step6], 'noname':[step4]}

# Ask the user for the name of the file to use
print(f"Available files:")
for file in file_options:
    print(f"> {file}")

user_input = input("Enter the name of the file to use: ")
# Find the matching file option
selected_file = None
for option in file_options:
    if user_input in option.lower():
        selected_file = option


# Check if the selected file exists
if selected_file:
    # Ask the user for the step to use
    print(f"Available steps for '{selected_file}':")
    for i, step in enumerate(file_options[selected_file]):
        print(f"> {i}. {get_var_name(step)}: {step}")

    user_step = int(input(f"Enter the INDEX number to use for '{selected_file}': "))
    try:
        selected_step = file_options[selected_file][user_step]


        # Iterate over the step parameters
        for i, param in enumerate(selected_step):
            if isinstance(param, list):
                if len(param) > 1: #if Choices
                    print(f"Available Parameter Choices")
                    for i2, choice in enumerate(param):
                      print(f"> {i2}. {choice}")
                    user_value = int(input("Choose the INDEX number for the parameter choice: "))
                    print("uservalue", user_value, param)
                    selected_step[i] = param[user_value]
                elif "value" in param[0]: #if value required
                    user_value = float(input(f"{param[0]}: "))
                    selected_step[i] = user_value
                else:
                    user_value = input(f"{param[0]}: ")
                    selected_step[i] = user_value
        print("\nThe input:", selected_step)
    except (IndexError):
        print("Invalid step number.")
else:
    print("Invalid file name.")

file_path = "DataSet/" + selected_file + ".dot"
print("Extracting the file....")
nodes, edges, adj_matrix = parse_dot_file(file_path)
structure, node_size, root, search_algorithm, search_root, directed, k, iterations, learning_rate, perplexity, episodes = selected_step
print("Drawing the graph....")
main(nodes, edges, structure, node_size, root, search_algorithm, search_root, directed, k, iterations, learning_rate, perplexity, episodes)